## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

#### если источник traffic_source равен yandex или google, то в source_type ставится organic
#### для источников paid и email из России - ставим ad
#### для источников paid и email не из России - ставим other
#### все остальные варианты берем из traffic_source без изменений

In [255]:
import pandas as pd

In [127]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [128]:
def source_type (row):
    '''Функция получает на вход строку дата фрейма, 
     ищет соответствия: источники paid и email из России, 
     если находит, записывает в новый столбец значение ad
     далее вполняет поиск: источники paid и email не из России, 
     если находит, записывает в новый столбец значение other
     для всех остальных записывает в новый столбец значения столбца traffic_source'''
    
    if 'Russia' in row['region'] and ('paid' in row['traffic_source'] or 'email' in row['traffic_source']):
        row['source_type'] = 'ad'
        
    elif 'Russia' not in row['region'] and ('paid' in row['traffic_source'] or 'email' in row['traffic_source']):
        row['source_type'] = 'other'
    
    else:
        row['source_type'] = row['traffic_source']
        
        return row
   
    return row  
        

In [129]:
#Запишем результат применения функции source_type к дф log в новый дф
log_source = log.apply(source_type, axis=1)
log_source.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,yandex


In [130]:
# при помощи метода str.contains для значений traffic_source = 'yandex|google' запишем 'organic' в source_type 
log_source.loc[log_source.traffic_source.str.contains('yandex|google'),'source_type'] = 'organic'
log_source.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [131]:
#выполним группировку по 'source_type' для проверки
log_sourse = log_source.groupby('source_type')
log_sourse.count()

,timestamp,visit_id,url,region,user_id,traffic_source
source_type,,,,,,
ad,3790,3790,3790,3790,3790,3790
direct,7013,7013,7013,7013,7013,7013
organic,5407,5407,5407,5407,5407,5407
other,2728,2728,2728,2728,2728,2728


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [205]:
df_url = pd.read_csv('URLs.txt ')

In [407]:
df_url.head(10)

,url,result_regul
0,/world/,False
1,/latest/,False
2,/?updated=top,False
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...,True
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...,True
6,/video/36001498-poyavilis-pervye-podrobnosti-g...,True
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True
8,/science/,False
9,/sport/,False


In [395]:
#при помощи метода str.contains и регулярного выражения '[/]\d{8}[-]*' найдем строки с соответствием шаблону
df_url['result_regul'] = df_url.url.str.contains('[/]\d{8}[-]*',regex=True) #запишем признак соответствия в новый столбец

In [396]:
df_url_ = df_url[df_url.result_regul == True].reset_index() # новый дф только с True
df_url_ 

,index,url,result_regul
0,3,/politics/36188461-s-marta-zhizn-rossiyan-susc...,True
1,4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True
2,5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...,True
3,6,/video/36001498-poyavilis-pervye-podrobnosti-g...,True
4,7,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True
...,...,...,...
79,88,/cis/35984145-kreml-prokommentiroval-soobschen...,True
80,89,/video/36071019-olimpiyskie-obekty-rio-prevrat...,True
81,90,/science/36151301-nazvano-posledstvie-zloupotr...,True
82,91,/incidents/36027330-vospitatelnitsu-zatravili-...,True


In [416]:
#запишем в отдельные столбцы заголовки и текст методом split
new_df = df_url_.url.str.split('[/]\d{8}[-]*', expand=True ).reset_index()

In [419]:
new_df.columns = ['index','title', 'news_text']#переименование столбцов в новой дф
new_df.head()

,index,title,news_text
0,0,/politics,s-marta-zhizn-rossiyan-suschestvenno-izmenitsy...
1,1,/world,tramp-pridumal-kak-reshit-ukrainskiy-vopros/
2,2,/science,nasa-sobiraet-ekstrennuyu-press-konferentsiyu-...
3,3,/video,poyavilis-pervye-podrobnosti-gibeli-natali-mel...
4,4,/world,tramp-pridumal-kak-reshit-ukrainskiy-vopros/?s...


In [413]:
new_df.groupby('title').count()

,index,news_text
title,,
/articles,7,7
/business,4,4
/cis,4,4
/crazy-world,2,2
/economics,3,3
/europe,1,1
/finances,1,1
/head,2,2
/health,2,2


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [304]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [305]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [306]:
#добавляем столбец с указанием даты и времени при помощи Встроенного метод pd.to_datetime для первода даты
ratings['time'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [307]:
#Посчитаем количество оценок, выданных каждым userId 
df_raitng = ratings.groupby('userId').count()

In [308]:
#Запишем в новый df данные только по тем userId, у кого количество оценок больше 100. 
df_raitng_count = df_raitng.loc[df_raitng.rating > 100].reset_index()

In [309]:
df_raitng_count

,userId,movieId,rating,timestamp,time
0,4,204,204,204,204
1,8,116,116,116,116
2,15,1700,1700,1700,1700
3,17,363,363,363,363
4,19,423,423,423,423
...,...,...,...,...,...
253,656,128,128,128,128
254,659,142,142,142,142
255,664,519,519,519,519
256,665,434,434,434,434


In [310]:
#замешем в новый дата фрейм данные по максимальному и минимальному значению time
live = ratings.groupby('userId').agg(['min', 'max'])['time'].reset_index()

In [311]:
live

,userId,min,max
0,1,2009-12-14 02:51:48,2009-12-14 02:53:25
1,2,1996-06-21 11:09:55,1996-06-21 11:24:06
2,3,2011-02-28 02:53:09,2011-02-28 22:39:47
3,4,2000-02-05 19:25:14,2000-02-08 03:57:54
4,5,2006-11-12 23:10:44,2006-11-12 23:45:45
...,...,...,...
666,667,1996-11-06 09:07:00,1996-11-06 09:23:54
667,668,2001-06-27 03:39:56,2001-06-27 03:44:38
668,669,2002-03-11 06:44:41,2002-03-11 06:56:06
669,670,1999-10-01 12:35:28,1999-10-26 13:20:33


In [312]:
#доавим в новый стобец 'live_user' рассчет разницы между мин и макс значение времени для каждого userId 
live['live_user'] = live.apply(lambda x: (x['max']-x['min']),axis=1)

In [313]:
live

,userId,min,max,live_user
0,1,2009-12-14 02:51:48,2009-12-14 02:53:25,0 days 00:01:37
1,2,1996-06-21 11:09:55,1996-06-21 11:24:06,0 days 00:14:11
2,3,2011-02-28 02:53:09,2011-02-28 22:39:47,0 days 19:46:38
3,4,2000-02-05 19:25:14,2000-02-08 03:57:54,2 days 08:32:40
4,5,2006-11-12 23:10:44,2006-11-12 23:45:45,0 days 00:35:01
...,...,...,...,...
666,667,1996-11-06 09:07:00,1996-11-06 09:23:54,0 days 00:16:54
667,668,2001-06-27 03:39:56,2001-06-27 03:44:38,0 days 00:04:42
668,669,2002-03-11 06:44:41,2002-03-11 06:56:06,0 days 00:11:25
669,670,1999-10-01 12:35:28,1999-10-26 13:20:33,25 days 00:45:05


In [322]:
#Объедимим результат в датафрейме joined
joined = df_raitng_count.merge(live, on='userId', how='left')
joined['live_user'].mean()

Timedelta('463 days 21:28:27.449612408')

## Задание 4:
Дана статистика услуг перевозок клиентов компании по типам .
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

In [323]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [324]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [325]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [326]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [331]:
# посредством merge outer объединим первые 2 таблицы
rzd_auto = rzd.merge(auto, how = 'outer', on = 'client_id')
rzd_auto

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [336]:
# посредством merge outer объединим полученный результат объединения с 3-м датафреймом
rzd_auto_air = rzd_auto.merge(air, how = 'outer', on = 'client_id')
rzd_auto_air #таблица с тремя типами выручки для каждого client_id

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [337]:
rzd_auto_air_addres= rzd_auto_air.merge(client_base, how = 'outer', on = 'client_id')
rzd_auto_air_addres #таблица по типам выручки с указанием адреса клиента

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
